## Aprendizado Supervisionado
### prof. Duncan
Exemplos com Naive Bayes Classifier

In [1]:
# atualizar pacote
#!pip install --upgrade scikit-learn
# pacotes básicos
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# pacotes do sklearn para acesso a datasets, preparação, modelagem e avaliação
from sklearn import datasets
# pacote pipeline para combinar preparação e modelagem
from sklearn.pipeline import Pipeline, make_pipeline
# arsenal de preparação
from sklearn.preprocessing import MinMaxScaler # rescala em min-max
from sklearn.preprocessing import StandardScaler # padroniza features removendo média e
#     escalando para variância unitária. Também chamado de z-score
#
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder #
from sklearn.preprocessing import OrdinalEncoder #
from sklearn.preprocessing import OneHotEncoder #
#
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import StratifiedKFold as skf
from sklearn.model_selection import GridSearchCV as gscv
from sklearn.naive_bayes import CategoricalNB as CNB
from sklearn.naive_bayes import GaussianNB as GNB
from sklearn.naive_bayes import BernoulliNB as BNB
from sklearn.metrics import accuracy_score as acc_score
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import ConfusionMatrixDisplay as CMD

#pacotes para apoio a leitura e gravação de datasets
from pathlib import Path
import csv

#pacotes para visualização e formatação
import pprint
import graphviz

# para silenciar warnings
import warnings
warnings.filterwarnings('ignore')

### Habilitar acesso ao google drive

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive/')

ModuleNotFoundError: No module named 'google'

### Leitura do dataset Breast Cancer

In [3]:
path_datasets = '/content/drive/My Drive/Datasets/Aprend_Super/'
arq = 'breast-cancer-2.csv'
breast_cancer = pd.read_csv(path_datasets + arq)
y = breast_cancer[['Class']]
X = breast_cancer.drop(['Class'], axis=1)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Datasets/Aprend_Super/breast-cancer-2.csv'

### Conteúdos do dataset

In [ ]:
X.info()

In [ ]:
X

In [ ]:
y.info()

### Protocolo experimental

In [ ]:
# separação entre treino e teste
treino_X, teste_X, treino_y, teste_y = tts(X, y, random_state=0,test_size=0.3, stratify=y)

print(treino_X.shape)
print(treino_y.shape)
print(teste_X.shape)
print(teste_y.shape)
print(np.stack(np.unique(teste_y, return_counts=True), axis=1))

In [ ]:
# colunas ordinais e nominais
col_nominais = ['menopause', 'breast','breast-quad']
col_ordinais = ['age', 'tumor-size','inv-nodes','node-caps','irradiat']
col_numericas = ['deg-malig']

In [ ]:
# preparação das colunas com ordinalencoder e onehotencoder
Prepara_Colunas = ColumnTransformer([
    ('Ordinal', OrdinalEncoder(dtype=int), col_ordinais),
    ('OneHot',  OneHotEncoder(dtype=int), col_nominais),
    ('MinMax',  MinMaxScaler(), col_numericas)
])
lbl21 = LabelEncoder()
Prepara_Colunas.fit(X)
w_X_n = pd.DataFrame(Prepara_Colunas.transform(treino_X), columns=Prepara_Colunas.get_feature_names_out())
w_y_n = lbl21.fit_transform(treino_y)
w_X_t_n = pd.DataFrame(Prepara_Colunas.transform(teste_X), columns=Prepara_Colunas.get_feature_names_out())
w_y_t_n = lbl21.transform(teste_y)

In [ ]:
Prepara_Colunas.feature_names_in_

In [ ]:
Prepara_Colunas.get_feature_names_out()

In [ ]:
all_features = Prepara_Colunas.get_feature_names_out()
numero_cols = len(all_features)
ord_features = all_features[:5]
num_features = all_features[numero_cols-1:numero_cols]
oho_features = all_features[5:numero_cols-1]

In [ ]:
# em Naive Bayes, os atributos são considerados independentes
# logo, pode-se calcular separadamente as probabilidades e, então,
# combiná-las

Alpha = 1.0e-10
modelo_ord = CNB(alpha=Alpha)
modelo_num = GNB()
modelo_oho = BNB()
modelo_ord.fit(w_X_n[ord_features], w_y_n)
modelo_num.fit(w_X_n[num_features], w_y_n)
modelo_oho.fit(w_X_n[oho_features], w_y_n)
w_y_predord_n = modelo_ord.predict(w_X_t_n[ord_features])
w_y_prednum_n = modelo_num.predict(w_X_t_n[num_features])
w_y_predoho_n = modelo_oho.predict(w_X_t_n[oho_features])
w_predord_proba = modelo_ord.predict_proba(w_X_t_n[ord_features])
w_prednum_proba = modelo_num.predict_proba(w_X_t_n[num_features])
w_predoho_proba = modelo_oho.predict_proba(w_X_t_n[oho_features])

# após cálculos separados, combinar os cálculos para a predição
w_pred_proba_aux = w_predord_proba * w_prednum_proba * w_predoho_proba
#print(w_pred_proba_aux)
w_pred_proba_aux2 = np.sum(w_pred_proba_aux, axis=1)
#print(w_pred_proba_aux2)
w_pred_proba = w_pred_proba_aux / w_pred_proba_aux2.reshape(-1, 1)
w_y_pred_n = np.argmax(w_pred_proba, axis=1)
acuracia = np.average(w_y_pred_n == w_y_t_n)

# listar resultados
resultado = cm(w_y_t_n, w_y_pred_n)
cm_display = CMD(resultado).plot()
print('Alpha={:5}  Acuracia={:.3}'.format(Alpha, acuracia))
#print(lbl21.classes_)
#print('Predição atributos categóricos:')
#print(w_predord_proba)
#print('Predição atributos numéricos:')
#print(w_prednum_proba)
#print('Predição atributos OneHot:')
#print(w_predoho_proba)
#print('Predição composta:')
#print(w_pred_proba)


### Naive Bayes com preparação como ordinais e execução do Categorical Naive Bayes

In [ ]:
#preparação todos como ordinais
categorias = [['20-29', '30-39', '40-49', '50-59', '60-69', '70-79'],
              ['premeno', 'lt40', 'ge40'],
              ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54'],
              ['0-2', '3-5', '6-8', '9-11', '12-14', '15-17', '18-20', '21-23', '24-26'],
              ['no', 'yes'],
              ['1', '2', '3'],
              ['left', 'right'],
              ['central', 'left_low', 'left_up', 'right_low', 'right_up'],
              ['no', 'yes']]

oe = OrdinalEncoder(dtype=int, categories=categorias)
oe.fit(X)
treino_X_n = pd.DataFrame(oe.transform(treino_X), columns=oe.get_feature_names_out())
teste_X_n = pd.DataFrame(oe.transform(teste_X), columns=oe.get_feature_names_out())
le = LabelEncoder()
le.fit(y)
treino_y_n = le.transform(treino_y)
teste_y_n = le.transform(teste_y)
#
print(treino_X_n.shape)
print(treino_y_n.shape)
print(teste_X_n.shape)
print(teste_y_n.shape)
print(np.stack(np.unique(teste_y_n, return_counts=True), axis=1))
# indução do modelo
Alpha = 1.0e-10
modelo = CNB(alpha=Alpha, min_categories=11)
modelo.fit(treino_X_n, treino_y_n)
teste_y_pred = modelo.predict(teste_X_n)
teste_y_pred_proba = modelo.predict_proba(teste_X_n)
acuracia = np.average(teste_y_pred == teste_y_n)
# listar resultados
resultado = cm(teste_y_n, teste_y_pred)
cm_display = CMD(resultado).plot()
print('Alpha={:5}  Acuracia={:.3}'.format(Alpha, acuracia))
print(lbl21.classes_)
#print('Predição:')
#print(teste_y_pred_proba)


### Naive Bayes com preparação como one hot encoding e execução com Bernoulli Naive Bayes

In [ ]:
#preparação todos como one hot
ohe = OneHotEncoder(sparse_output=False, dtype=int)
ohe.fit(X)
treino_X_b = pd.DataFrame(ohe.transform(treino_X), columns=ohe.get_feature_names_out())
teste_X_b = pd.DataFrame(ohe.transform(teste_X), columns=ohe.get_feature_names_out())
le = LabelEncoder()
le.fit(y)
treino_y_b = le.transform(treino_y)
teste_y_b = le.transform(teste_y)
# indução do modelo
modeloB = BNB(alpha=Alpha)
modeloB.fit(treino_X_b, treino_y_b)
teste_y_b_pred = modeloB.predict(teste_X_b)
teste_y_b_pred_proba = modeloB.predict_proba(teste_X_b)
acuracia = np.average(teste_y_b_pred == teste_y_b)
# listar resultados
resultado = cm(teste_y_b, teste_y_b_pred)
cm_display = CMD(resultado).plot()
print('Alpha={:5}  Acuracia={:.3}'.format(Alpha, acuracia))
print(lbl21.classes_)
#print('Predição:')
#print(teste_y_pred_proba)
